<a href="https://colab.research.google.com/github/grimtoma/book_recomandation/blob/main/recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise - Data Scientist
Problem: Book recommendations - „I like Lord of the Rings, what else should I read?“

**Goal:**
- take some data, try some approaches, produce some code, get some results
- then come and show us your solution and have a chat around it - show how you think about a specific problem, how you are able to explain what approach you used and why, think about the limitations of the approach and how things could be improved if there was more time, what you think of the results and if they make sense, etc.

**Philosophy:**
- the actual result and code are not that important – the journey there and potential future paths are more important
- if you are able to follow-through with some ideas then great, if you just start something and have a clear idea on how to proceed that is also useful
- the expectation is that you will spend an evening or two with the task (but there are no bounds to proactivity if you enjoy playing with the problem)

**Data:**
- available open dataset: http://www2.informatik.uni-freiburg.de/~cziegler/BX/
- alternatively feel free to grab any other relevant data set

**Tools:**
- use whatever you are comfortable in (R, Python, Matlab, Java, SQL,…) or feel free to use it as an opportunity to try out a new language
- it is not a contest in finding the best black-box library and blindly using it – own solutions are preferred even if they are simple
- how to present: up to you – slides, walking through code, drawing, …